<a href="https://colab.research.google.com/github/vishal-vichu14/Multi-Label-Classification-of-Customer-Support-Tickets-with-Fine-Tuned-BERT/blob/main/step3ofBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# Load the dataset
final_data = pd.read_csv('/content/final_data (1).xls')
final_data.head()

,reviewText,team_name
0,"Well, after trying out some Box Towers....that...",Audio Systems Team
1,I ordered one for my wife and one for me. Afte...,Imaging and Optics Team
2,muy buen producto... a full en juegos fHD.... ...,Customer Experience Team
3,The sound quality of this unit is phenomenal. ...,Audio Systems Team
4,It is good on keeping your cpu cool also down'...,Customer Experience Team


In [ ]:
final_data['reviewText'][1]

"I ordered one for my wife and one for me. After reading all the negative reviews on picture quality, I almost returned them on arrival. But I didn't and both me and my wife weer pleasantly amazed at the ease of use and high quality pictures we got immediately. I suspect people play with the settings and especially the high iso one and end of with less resolution for low light which may not be needed anyway. The auto function works extemely well as it select the best scene, focus and shutter. All our initial pictures were just perfect, even under less than idea conditions. For my book I give it an overall 4.5, and that is just because the camera is still new and still learning the full features, I would recommencd it as others have. None were blurred."

In [ ]:
# Extract unique team names to create label mappings
labels = final_data['team_name'].unique().tolist()

In [ ]:
labels

['Audio Systems Team',
 'Imaging and Optics Team',
 'Customer Experience Team',
 'Protection and Installation Team',
 'Power and Battery Solutions',
 'Peripheral Devices Team',
 'Connectivity Solutions Team']

In [ ]:
num_labels = len(labels)
# Create dictionaries for mapping between label IDs and label names
id2label = {id:label for id,label in enumerate(labels)}
label2id = {label:id for id,label in enumerate(labels)}

In [ ]:
id2label

{0: 'Audio Systems Team',
 1: 'Imaging and Optics Team',
 2: 'Customer Experience Team',
 3: 'Protection and Installation Team',
 4: 'Power and Battery Solutions',
 5: 'Peripheral Devices Team',
 6: 'Connectivity Solutions Team'}

In [ ]:
label2id

{'Audio Systems Team': 0,
 'Imaging and Optics Team': 1,
 'Customer Experience Team': 2,
 'Protection and Installation Team': 3,
 'Power and Battery Solutions': 4,
 'Peripheral Devices Team': 5,
 'Connectivity Solutions Team': 6}

In [ ]:
# Assign numeric labels to each team name in the dataset
final_data['labels'] = pd.factorize(final_data.team_name)[0]

In [ ]:
final_data.head()

,reviewText,team_name,labels
0,"Well, after trying out some Box Towers....that...",Audio Systems Team,0
1,I ordered one for my wife and one for me. Afte...,Imaging and Optics Team,1
2,muy buen producto... a full en juegos fHD.... ...,Customer Experience Team,2
3,The sound quality of this unit is phenomenal. ...,Audio Systems Team,0
4,It is good on keeping your cpu cool also down'...,Customer Experience Team,2


In [ ]:
# Import the necessary components from the Transformers library
from transformers import BertTokenizer,BertForSequenceClassification

In [ ]:
# Load the pre-trained BERT tokenizer and model for sequence classification
tokenizer = tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = num_labels ,id2label = id2label,label2id = label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set up the device to use GPU if available, otherwise fallback to CPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Convert review text to string format
final_data['reviewText'] = final_data['reviewText'].astype(str)

In [ ]:
# Determine the size of the dataset
SIZE = final_data.shape[0]
# Split the dataset into training, validation, and test sets
train_texts = list(final_data.reviewText[:SIZE//2])
val_texts = list(final_data.reviewText[SIZE//2:(3*SIZE)//4])
test_texts = list(final_data.reviewText[(3*SIZE)//4:])
# Split the labels accordingly
train_labels = list(final_data.labels[:SIZE//2])
val_labels = list(final_data.labels[SIZE//2:(3*SIZE)//4])
test_labels = list(final_data.labels[(3*SIZE)//4:])

In [ ]:
# Function to tokenize the text data
def tokenize_data(text_list):
    return tokenizer(
        text_list,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

In [ ]:
# Tokenize the training, validation, and test sets
train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)
test_encodings = tokenize_data(test_texts)

In [ ]:
# Import necessary components for creating a DataLoader
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

In [ ]:
# Custom dataset class for handling the tokenized data and labels
class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
      # Convert encodings to tensors and return with labels
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



In [ ]:
# Create DataLoader objects for the training, validation, and test datasets
train_dataloader = ReviewsDataset(train_encodings,train_labels)
val_dataloader = ReviewsDataset(val_encodings,val_labels)
test_dataloader = ReviewsDataset(test_encodings,test_labels)

In [ ]:
# Import necessary components for training
from transformers import TrainingArguments, Trainer

In [ ]:
# Function to compute evaluation metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ =precision_recall_fscore_support(labels,preds,average='macro')
    acc = accuracy_score(labels,preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./TTC4900MODEL',  # Directory to save model checkpoints
    do_train=True,
    do_eval=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=32,  # Batch size for evaluation
    warmup_steps=100,  # Warmup steps for learning rate scheduler
    weight_decay=0.01,  # Weight decay for optimizer
    logging_strategy="steps",
    logging_dir='./multi-class-logs',  # Directory for logs
    logging_steps=50,  # Log every 50 steps
    eval_strategy="steps",  # Evaluation strategy
    eval_steps=50,  # Evaluate every 50 steps
    save_strategy="steps",  # Save model every 50 steps
    fp16=True,  # Use mixed precision training (float16)
    load_best_model_at_end=True  # Load the best model at the end of training
)

In [ ]:
# Initialize the Trainer object with the model, training arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics = compute_metrics
    )

In [ ]:
# Train the model
trainer.train()

<ipython-input-19-80911b05bc00>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.730200,1.558740,0.490000,0.093960,0.070000,0.142857
100,1.442000,1.043155,0.654400,0.550811,0.734250,0.580999
150,0.736800,0.618744,0.820800,0.798021,0.805616,0.808669
200,0.509900,0.550120,0.824000,0.814429,0.781849,0.870273
250,0.532300,0.488736,0.850400,0.838116,0.819905,0.862625
300,0.470900,0.575737,0.824800,0.823468,0.781056,0.887202
350,0.334600,0.497934,0.853600,0.850707,0.826051,0.881383
400,0.338400,0.541446,0.849200,0.835901,0.814579,0.870431
450,0.353900,0.641796,0.808000,0.805534,0.766273,0.867306
500,0.379800,0.438837,0.856000,0.847024,0.837762,0.861182


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-19-80911b05bc00>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-19-80911b05bc00>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-19-80911b05bc00>:7: UserWarning: To copy construct from

TrainOutput(global_step=939, training_loss=0.45122346481956993, metrics={'train_runtime': 731.4176, 'train_samples_per_second': 20.508, 'train_steps_per_second': 1.284, 'total_flos': 3946843008000000.0, 'train_loss': 0.45122346481956993, 'epoch': 3.0})

In [ ]:
q = [trainer.evaluate(eval_dataset=final_data)for final_data in [train_dataloader,val_dataloader,test_dataloader]]

<ipython-input-19-80911b05bc00>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<ipython-input-19-80911b05bc00>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-19-80911b05bc00>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.206213,0.9430,0.942969,0.941198,0.945625
val,0.438837,0.8560,0.847024,0.837762,0.861182
test,0.400173,0.8692,0.861134,0.857020,0.869533


In [ ]:
# Evaluate the model on training, validation, and test datasets
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

In [ ]:
# Function to make predictions on new text data
def predict(text):
  inputs = tokenizer(text, padding = True, truncation = True, max_length = 512, return_tensors ="pt").to('cuda')
  output = model(**inputs)
  probs = output[0].softmax(1)
  pred_label_idx = probs.argmax()
  pred_label = model.config.id2label[pred_label_idx.item()]
  return probs,pred_label_idx,pred_label

In [ ]:
# Save the trained model and tokenizer for later use
model_path = "multi-class-model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('multi-class-model/tokenizer_config.json',
 'multi-class-model/special_tokens_map.json',
 'multi-class-model/vocab.txt',
 'multi-class-model/added_tokens.json')

In [ ]:
# Load the model and tokenizer from the saved files
from transformers import pipeline
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer,device=0)

In [ ]:
# Create a pipeline for sentiment analysis (can be used for predictions)
nlp("I ordered one for my wife and one for me. After reading all the negative reviews on picture quality, I almost returned them on arrival. But I didn't and both me and my wife weer pleasantly amazed at the ease of use and high quality pictures we got immediately. I suspect people play with the settings and especially the high iso one and end of with less resolution for low light which may not be needed anyway. The auto function works extemely well as it select the best scene, focus and shutter. All our initial pictures were just perfect, even under less than idea conditions. For my book I give it an overall 4.5, and that is just because the camera is still new and still learning the full features, I would recommencd it as others have. None were blurred")

[{'label': 'Imaging and Optics Team', 'score': 0.8903094530105591}]

In [ ]:
nlp("I brought a phone, which is very nice but it's battery performance is very poor")

[{'label': 'Power and Battery Solutions', 'score': 0.9295383095741272}]

In [ ]:
nlp("The camera mount i brought is very bad and worst quality")

[{'label': 'Imaging and Optics Team', 'score': 0.9445143938064575}]

In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install streamlit-option-menu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 15.1 MB/s eta 0:00:00


In [ ]:
!pip install streamlit-lottie

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 30.6 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
! streamlit run page.py & npx localtunnel --port 8501